In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open("names.txt", "r").read().splitlines()
chars = sorted(list(set(''.join(words))))
stoi = {s:i for i, s in enumerate(["."]+chars)}
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)
block_size = 9 # context length

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])   # training
Xdev, Ydev = build_dataset(words[n1:n2]) # dev
Xte,  Yte  = build_dataset(words[n2:])   # test

{0: '.', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
27
torch.Size([182625, 9]) torch.Size([182625])
torch.Size([22655, 9]) torch.Size([22655])
torch.Size([22866, 9]) torch.Size([22866])


In [4]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate : {str(app):5s} | maxdiff: {maxdiff}')

In [5]:
# mlp
n_embed = 10 # dimensionality of the character embedding
n_hidden = 200 # n neurons in hidden layer of MLP

g  = torch.Generator().manual_seed(2147483647)
C  = torch.randn((vocab_size, n_embed), generator=g)          # vocabulary lookup into embedding
# the "kaiming-init"
W1 = torch.randn((n_embed*block_size, n_hidden), generator=g) *(5/3)/(n_embed*block_size)**.5 # * 0.2 # embedding into latent space
b1 = torch.randn(n_hidden, generator=g) * 0.1                      # latent space bias
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1   # embedding into vocabulary one-hot
b2 = torch.randn(vocab_size, generator=g) * 0.1                 # vocabulary bias

bngain = torch.ones((1, n_hidden))*0.1+1.0
bnbias = torch.zeros((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True
# why can't you set W2 to zero to get that expected value?
# tbd...

24297


In [6]:
batch_size = 32
n = batch_size
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [7]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.7248, grad_fn=<NegBackward0>)

In [10]:
W2.T.shape

torch.Size([27, 200])

In [11]:
dlogits.shape

torch.Size([32, 27])

In [15]:
b2.shape

torch.Size([27])

In [20]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n

dprobs = probs**(-1) * dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(dim=1, keepdims=True)

dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv

dcounts = counts_sum_inv * dprobs + dcounts_sum

dnorm_logits = counts * dcounts

dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(dim=1, keepdims=True)
# dlogit_max is propagated only into index that has the actual argmax
dlogits += torch.scatter(torch.zeros_like(dlogits), 1, torch.argmax(logits, dim=1, keepdim=True), dlogit_maxes)

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)

In [21]:
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate : True  | maxdiff: 0.0
probs           | exact: True  | approximate : True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate : True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate : True  | maxdiff: 0.0
counts          | exact: True  | approximate : True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate : True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate : True  | maxdiff: 0.0
logits          | exact: True  | approximate : True  | maxdiff: 0.0
h               | exact: True  | approximate : True  | maxdiff: 0.0
W2              | exact: True  | approximate : True  | maxdiff: 0.0
b2              | exact: True  | approximate : True  | maxdiff: 0.0


NameError: name 'dhpreact' is not defined